<h1> Simple Tensorflow Prediction Pipeline </h1>
The goal of this notebook is to serve as an example for a tensorflow model. I simplified many steps and ignored some other steps to create an understandable pipeline. This is also my first tensorflow model, so tips are welcome!

<h3> A full pipeline would have the the following elements: </h3>
1. Normalize colors of images -> Ignored for now. Could be an important performance booster in the future
2. Resize images
3. Prepare the X and Y matrices
4. A. Define a tensorflow graph model for region proposal
4. B. Define a tensorflow graph model for pixel segmentation
4. C. Combine the output of models A and B
5. Define the IOU metric. 
6. Run the model and evaluate the cost.

<h3> The current version of this notebook covers te following elements </h3>
1. Prepare X and Y matrices: 50 samples of  256x256x4 images, 40 for training, 10 for validation
2. Define a tensorflow graph model (convolutional) to find the difference between nucleus (1) and background (0)
3. Define an IOU metric
4. Run the model and evaluate the cost

But first: let's import the packages.

In [ ]:
"""IMPORT"""
import numpy as np
import pandas as pd
import os
from skimage.io import imread
import matplotlib.pyplot as plt
import gc
import tensorflow as tf
from PIL import Image
from subprocess import check_output
import time
#print(check_output(["ls", "../input"]).decode("utf8"))

"""PREPARATION"""
train_path = r"../input/stage1_train/"
train_ids = os.listdir(train_path)
subfolders = ["images","masks"]
train_list = []


"""We want 50 256x256x4 images"""
shape = 256, 256, 4
m = 50

 <h1> 1. Prepare X and Y matrices </h1>
 Constraints:
* Only 256x256x4 pictures are used because we skip the resizing proces
* We limit the amount of training examples to 50 to save processing time

In [ ]:
def get_XY(shape, m, train_path, train_ids):
    """
    Description: 
        This function loops over the training ID's and masks to select m images and labels that match the 
        desired shape and return it. 
    Input: 
        - shape: dimensions of the picture (height x width x channels), a selection criterion
        - m: number of samples, a selection criterion
        - train_path: path of the train folder
        - train_ids: list of training Ids.
    Output: 
        - X: A stack of images. dimension is  (m x shape) where shape is (height x width x channels)
        - Y: A stack of labels to predict. sum of all masks for that object
    """
    i=0  # keep track of number of images retrieved
    init_Y = False  # define whether Y is initialized
    init_X = False  # define whether X is initialized
    
    #  Loop over the training id's.
    for train_id in train_ids:
        
        # Load an image
        x_path = train_path + train_id + "/images/" + train_id +  ".png"
        x = Image.open(x_path)
        x = np.array(x)
        
        # Check if the size matches the expected size, else ignore 
        if shape != x.shape:
            continue 
        
        # Add an axis to stack the samples along. 
        x = np.expand_dims(x,0)
        
        # Stack the training examples
        if not init_X:
            X = x
            init_X = True
        else:
            X = np.vstack([X,x])
    
        # Load a mask
        y_path = train_path + train_id + "/masks/" 
        maskpaths = os.listdir(y_path)

        # A single image id has multiple masks -> sum over the masks to create 1 label per image id.
        init_y = False
        for maskpath in maskpaths:
            temppath = y_path + maskpath
            if not init_y:
                y = Image.open(temppath)
                y = np.array(y)
                init_y = True
            else:
                temp = Image.open(temppath)
                temp = np.array(temp)
                y = y + temp       

        # Add an axis to stack the samples along.       
        y = np.expand_dims(y,0)
        print(y.shape)
        if not init_Y:
            Y = y
            init_Y = True
        else:
            Y = np.vstack([Y,y])
        i+=1
        print(i)
        if i >= m:
            print("sampling limit reached.")
            Y = np.expand_dims(Y,3)
            Y = Y/255
            X = X/255
            return X,Y

In [ ]:
X,Y = get_XY(shape, m, train_path, train_ids)

In [ ]:
# check the shapes
print("X.shape = ",X.shape)
print("Y.shape = ",Y.shape)

In [ ]:
# check a sample 
plt.subplot(121)
plt.imshow(X[0])
plt.title("Image")
plt.subplot(122)
plt.imshow(np.squeeze(Y[0]))
plt.title("mask")

<h1> 2. Tensorflow graph model </h1>
Here I'll build a convolutional neural network in tensorflow

<h3> Placeholders </h3>

In [ ]:
# Placeholders can be used to define the computations graph
def get_placeholders(size):
    Xtf = tf.placeholder(name="Xtf",shape=[None,size[0],size[1],4],dtype="float")
    Ytf = tf.placeholder(name="Ytf",shape=[None,size[0],size[1],1],dtype="float")
    return Xtf, Ytf

<h3> The computation graph </h3>

In [ ]:
def go_forward(Xtf):
    
    # PARAMETERS
    W1 = tf.get_variable("W1",shape=[3,3,4,32],initializer=tf.contrib.layers.xavier_initializer(seed=0),dtype="float")
    W2 = tf.get_variable("W2",shape=[3,3,32,64],initializer=tf.contrib.layers.xavier_initializer(seed=0),dtype="float")
    W3 = tf.get_variable("W3",shape=[3,3,64,128],initializer=tf.contrib.layers.xavier_initializer(seed=0),dtype="float")
    W4 = tf.get_variable("W4",shape=[3,3,128,1],initializer=tf.contrib.layers.xavier_initializer(seed=0),dtype="float")

    # LAYER 1
    Z1 = tf.nn.conv2d(Xtf,W1,strides=[1,1,1,1],padding="SAME")
    A1 = tf.nn.relu(Z1)
    
    # LAYER 2
    Z2 = tf.nn.conv2d(A1,W2,strides=[1,1,1,1],padding="SAME")
    A2 = tf.nn.relu(Z2)
    
    # LAYER 3
    Z3 = tf.nn.conv2d(A2,W3,strides=[1,1,1,1],padding="SAME")
    A3 = tf.nn.relu(Z3)
    
    # LAYER 4
    Z4 = tf.nn.conv2d(A3,W4,strides=[1,1,1,1],padding="SAME")
    A4 = tf.nn.sigmoid(Z4)
    
    return A4

<h1>3. The IOU metric</h1>

In [ ]:
def compute_cost(A4, Y):
    print("A4 ",A4.shape)
    Y_negatives = tf.subtract(1.,Y)
    print("Y_neg ",Y_negatives)
    tp = tf.reduce_sum(tf.reduce_sum(tf.multiply(A4,Y),1),1)  # True positives (i.e. the intersection)
    print("true_pos ",tp)
    fp = tf.reduce_sum(tf.reduce_sum(tf.multiply(A4,Y_negatives),1),1)  # False positives
    fn = tf.reduce_sum(tf.reduce_sum(tf.multiply(tf.subtract(Y,A4),Y),1),1)  # False negatives
    un_temp = tf.add(tp,fp)  # sum of true and false positvives
    un = tf.add(un_temp,fn)  # The union
    iou = tf.divide(tp,un)  # the intersection over union
    cost = tf.subtract(1.,tf.reduce_mean(iou))  # the cost
    return (cost, tp, fp, fn)

<h1>4. Running the model </h1>

In [ ]:
tf.reset_default_graph()
best_score = 1.
best_iter = 0
start_time = time.time()
max_duration = 900
max_iterations = 80
X_train = X[:-10]
X_valid = X[-10:]
Y_train = Y[:-10]
Y_valid = Y[-10:]
train_costs = [1] * max_iterations
valid_costs = [1] * max_iterations
with tf.Session() as sess:    
    Xtf, Ytf = get_placeholders(shape)
    A4 = go_forward(Xtf)
    cost, tp, fp, fn = compute_cost(A4,Ytf)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.002).minimize(cost)
    init = tf.global_variables_initializer()
    sess.run(init)
    for i in range(max_iterations):
        pred, train_costs[i],tpo, fpo, fno = sess.run([optimizer,cost, tp, fp, fn],feed_dict={Xtf:X_train,Ytf:Y_train})
        valid_costs[i] = sess.run(cost,feed_dict={Xtf:X_valid,Ytf:Y_valid})

        print("\n************************")
        print("iter:",i)
        print("training cost:",train_costs[i])
        print("valid cost:",valid_costs[i])
        print("************************\n")
        
        if train_costs[i] > best_score and best_iter <= i-10:
            print("Converged")
            break
        elif time.time() >= start_time + max_duration:
            print("timed out")
            break
        else:
            best_score = train_costs[i]
            best_iter = i
    prediction, iou_pred = sess.run([A4,cost],{Xtf:X_valid,Ytf:Y_valid})
    print("validation cost: ",iou_pred)
#%%  CHECK OUTPUT

<h3> The learning curve </h3>

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(range(i),train_costs[:i],label="neg. training IOU")
plt.plot(range(i),valid_costs[:i],label="neg. validation IOU",c='r')

 <h3> Visualization </h3>

In [ ]:
def show(out,Y,i=0):
    fig = plt.figure(figsize=(10,15))
        

    ax_1 = fig.add_subplot(1,2,1)
    ax_1.set_title("Predicted mask " + str(i),fontsize=12)
    ax_1.imshow(np.squeeze(out[i]),cmap="Greys")
    
    ax_2 = fig.add_subplot(1,2,2)
    ax_2.set_title("Actual mask " + str(i),fontsize=12)
    ax_2.imshow(np.squeeze(Y[i]),cmap="Greys")

    plt.show()

for j in range(10):
    show(prediction,Y_valid,i=j)